## importing necessary libs

i'm using poetry for managing dependencies and my virtual env, so i'd recommend to look `pyproject.toml` to know what's been used here.

In [1]:
import findspark
from pyspark import SparkContext
from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt

## setting up spark session and dataframe

In [ ]:
#https://spark.apache.org/docs/latest/sql-getting-started.html
spark = SparkSession \
        .builder \
        .appName("firstSpark") \
        .getOrCreate()

In [3]:
# loading dataframe
def load_dataframe(filename):
    df = spark.read.format('csv').options(header='true').load(filename)
    return df

#creating a dataframe
df_matches = load_dataframe('./data/matches.csv')
df_matches.limit(5).show()

+--------+---+------+----------+-------------+--------------+----+----+---+
|Match_ID|Div|Season|      Date|     HomeTeam|      AwayTeam|FTHG|FTAG|FTR|
+--------+---+------+----------+-------------+--------------+----+----+---+
|       1| D2|  2009|2010-04-04|   Oberhausen|Kaiserslautern|   2|   1|  H|
|       2| D2|  2009|2009-11-01|  Munich 1860|Kaiserslautern|   0|   1|  A|
|       3| D2|  2009|2009-10-04|Frankfurt FSV|Kaiserslautern|   1|   1|  D|
|       4| D2|  2009|2010-02-21|Frankfurt FSV|     Karlsruhe|   2|   1|  H|
|       5| D2|  2009|2009-12-06|        Ahlen|     Karlsruhe|   1|   3|  A|
+--------+---+------+----------+-------------+--------------+----+----+---+



In [4]:
# im doing this just to understand zip command

slice_a = 'abcdefg'
slice_b = range(3)
slice_c = 'vwxyz'

print(f'lets go! \nslices:\n\t{slice_a}\n\t{list(slice_b)}\n\t{slice_c}')

# so if i use zip i expect a list with the array values intercalated, until the shorter list reachs its end
print(f'ZIP doing magic!\n\t{list(zip(slice_a, slice_b, slice_c))}') # got what was expected, yay!

# now im using starred expressions used for unpacking lists
unpacked = [*zip(slice_a, slice_b, slice_c)]
print(f'STARRED ZIP doing magic!\n\t{unpacked}') # need to practice more of this, seems useful

lets go! 
slices:
	abcdefg
	[0, 1, 2]
	vwxyz
ZIP doing magic!
	[('a', 0, 'v'), ('b', 1, 'w'), ('c', 2, 'x')]
STARRED ZIP doing magic!
	[('a', 0, 'v'), ('b', 1, 'w'), ('c', 2, 'x')]


In [5]:
# lets rename some of the columns (FTHG, FTAG, FTR)
old_cols = df_matches.columns[-3:]
new_cols = ["HomeTeamGoals", "AwayTeamGoals", "FinalResult"]
old_new_cols = [*zip(old_cols, new_cols)]
for old_col, new_col in old_new_cols:
    df_matches = df_matches.withColumnRenamed(old_col, new_col)

df_matches.limit(5).toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,FinalResult
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


## shine pyspark!

### 1) Who are the winners of the D1 division in the Germany Football Association (Bundesliga) between 2000–2010?

In [6]:
df_matches = df_matches \
    .withColumn('HomeTeamWin', when(col('FinalResult') == 'H', 1).otherwise(0)) \
    .withColumn('AwayTeamWin', when(col('FinalResult') == 'A', 1).otherwise(0)) \
    .withColumn('GameTie', when(col('FinalResult') == 'D', 1).otherwise(0))


#bundesliga is a D1 division and we are interested in season <= 2010 and >= 2000
bundesliga = df_matches \
                    .filter((col('Season') >= 2000) & 
                            (col('Season') <= 2010) & 
                            (col('Div') == 'D1'))

# home team features
home = bundesliga.groupby('Season', 'HomeTeam') \
       .agg(sum('HomeTeamWin').alias('TotalHomeWin'),
            sum('AwayTeamWin').alias('TotalHomeLoss'),
            sum('GameTie').alias('TotalHomeTie'),
            sum('HomeTeamGoals').alias('HomeScoredGoals'),
            sum('AwayTeamGoals').alias('HomeAgainstGoals')) \
       .withColumnRenamed('HomeTeam', 'Team')

#away game features 
away =  bundesliga.groupby('Season', 'AwayTeam') \
       .agg(sum('AwayTeamWin').alias('TotalAwayWin'),
            sum('HomeTeamWin').alias('TotalAwayLoss'),
            sum('GameTie').alias('TotalAwayTie'),
            sum('AwayTeamGoals').alias('AwayScoredGoals'),
            sum('HomeTeamGoals').alias('AwayAgainstGoals'))  \
       .withColumnRenamed('AwayTeam', 'Team')

#### seeing both results

In [7]:
home.toPandas()

,Season,Team,TotalHomeWin,TotalHomeLoss,TotalHomeTie,HomeScoredGoals,HomeAgainstGoals
0,2005,Kaiserslautern,5,7,5,26.0,33.0
1,2006,Cottbus,6,6,5,21.0,22.0
2,2001,St Pauli,4,9,4,19.0,28.0
3,2005,Mainz,6,4,7,31.0,23.0
4,2006,Hamburg,4,4,9,22.0,19.0
...,...,...,...,...,...,...,...
193,2003,Kaiserslautern,8,4,5,25.0,19.0
194,2004,Bielefeld,7,7,3,21.0,21.0
195,2004,Werder Bremen,9,4,4,33.0,15.0
196,2004,Stuttgart,12,3,2,34.0,15.0


In [8]:
away.toPandas()

,Season,Team,TotalAwayWin,TotalAwayLoss,TotalAwayTie,AwayScoredGoals,AwayAgainstGoals
0,2005,Kaiserslautern,3,10,4,21.0,38.0
1,2006,Cottbus,5,9,3,17.0,27.0
2,2001,St Pauli,0,11,6,18.0,42.0
3,2005,Mainz,3,10,4,15.0,24.0
4,2006,Hamburg,6,5,6,21.0,18.0
...,...,...,...,...,...,...,...
193,2003,Kaiserslautern,3,13,1,14.0,43.0
194,2004,Bielefeld,4,9,4,16.0,28.0
195,2004,Werder Bremen,9,7,1,35.0,22.0
196,2004,Stuttgart,5,7,5,20.0,25.0


#### answering the question

In [9]:
# it seems window turns easy to use some aggregation functions like rank and statistical analysis methods
# https://sparkbyexamples.com/pyspark/pyspark-window-functions/
window = ['Season']
window = Window.partitionBy(window).orderBy(col('WinPct').desc(), col('GoalDifferentials').desc())
table = home.join(away, ['Team', 'Season'],  'inner') \
    .withColumn('GoalsScored', col('HomeScoredGoals') + col('AwayScoredGoals')) \
    .withColumn('GoalsAgainst', col('HomeAgainstGoals') + col('AwayAgainstGoals')) \
    .withColumn('GoalDifferentials', col('GoalsScored') - col('GoalsAgainst')) \
    .withColumn('Win', col('TotalHomeWin') + col('TotalAwayWin')) \
    .withColumn('Loss', col('TotalHomeLoss') + col('TotalAwayLoss')) \
    .withColumn('Tie', col('TotalHomeTie') + col('TotalAwayTie')) \
    .withColumn('WinPct', round((100* col('Win')/(col('Win') + col('Loss') + col('Tie'))), 2)) \
    .drop('HomeScoredGoals', 'AwayScoredGoals', 'HomeAgainstGoals', 'AwayAgainstGoals') \
    .drop('TotalHomeWin', 'TotalAwayWin', 'TotalHomeLoss', 'TotalAwayLoss', 'TotalHomeTie', 'TotalAwayTie') \
    .withColumn('TeamPosition', rank().over(window)) 

table_df = table.filter(col('TeamPosition') == 1).orderBy(desc('Season')).toPandas()
table_df

,Team,Season,GoalsScored,GoalsAgainst,GoalDifferentials,Win,Loss,Tie,WinPct,TeamPosition
0,Dortmund,2010,67.0,22.0,45.0,23,5,6,67.65,1
1,Bayern Munich,2009,72.0,31.0,41.0,20,4,10,58.82,1
2,Wolfsburg,2008,80.0,41.0,39.0,21,7,6,61.76,1
3,Bayern Munich,2007,68.0,21.0,47.0,22,2,10,64.71,1
4,Stuttgart,2006,61.0,37.0,24.0,21,6,7,61.76,1
5,Bayern Munich,2005,67.0,32.0,35.0,22,3,9,64.71,1
6,Bayern Munich,2004,75.0,33.0,42.0,24,5,5,70.59,1
7,Werder Bremen,2003,79.0,38.0,41.0,22,4,8,64.71,1
8,Bayern Munich,2002,70.0,25.0,45.0,23,5,6,67.65,1
9,Leverkusen,2001,77.0,38.0,39.0,21,7,6,61.76,1


### 2) Which teams have been relegated in the past 10 years?

In [22]:
bundesliga_relegated = df_matches \
                        .filter((col('Season') >= 2007) & 
                                (col('Season') <= 2017) & 
                                (col('Div') == 'D1'))

home = bundesliga_relegated.groupby('Season', 'HomeTeam') \
       .agg(sum('HomeTeamWin').alias('TotalHomeWin'),
            sum('AwayTeamWin').alias('TotalHomeLoss'),
            sum('GameTie').alias('TotalHomeTie'),
            sum('HomeTeamGoals').alias('HomeScoredGoals'),
            sum('AwayTeamGoals').alias('HomeAgainstGoals')) \
       .withColumnRenamed('HomeTeam', 'Team')

away = bundesliga_relegated.groupby('Season', 'AwayTeam') \
       .agg(sum('AwayTeamWin').alias('TotalAwayWin'),
            sum('HomeTeamWin').alias('TotalAwayLoss'),
            sum('GameTie').alias('TotalAwayTie'),
            sum('AwayTeamGoals').alias('AwayScoredGoals'),
            sum('HomeTeamGoals').alias('AwayAgainstGoals')) \
       .withColumnRenamed('AwayTeam', 'Team')

#### seeing both results

In [23]:
home.sort('Season').toPandas()

,Season,Team,TotalHomeWin,TotalHomeLoss,TotalHomeTie,HomeScoredGoals,HomeAgainstGoals
0,2007,Hansa Rostock,5,8,4,17.0,21.0
1,2007,Cottbus,8,7,2,25.0,20.0
2,2007,Bochum,5,3,9,32.0,28.0
3,2007,Ein Frankfurt,8,5,4,24.0,24.0
4,2007,Stuttgart,12,3,2,39.0,19.0
...,...,...,...,...,...,...,...
193,2017,FC Koln,17,0,0,-17.0,-17.0
194,2017,Wolfsburg,17,0,0,-17.0,-17.0
195,2017,Dortmund,17,0,0,-17.0,-17.0
196,2017,Freiburg,17,0,0,-17.0,-17.0


In [24]:
away.sort('Season').toPandas()

,Season,Team,TotalAwayWin,TotalAwayLoss,TotalAwayTie,AwayScoredGoals,AwayAgainstGoals
0,2007,Hansa Rostock,3,12,2,13.0,31.0
1,2007,Cottbus,1,9,7,10.0,36.0
2,2007,Bochum,5,10,2,16.0,26.0
3,2007,Ein Frankfurt,4,7,6,19.0,26.0
4,2007,Stuttgart,4,11,2,18.0,38.0
...,...,...,...,...,...,...,...
193,2017,FC Koln,0,17,0,-17.0,-17.0
194,2017,Wolfsburg,0,17,0,-17.0,-17.0
195,2017,Dortmund,0,17,0,-17.0,-17.0
196,2017,Freiburg,0,17,0,-17.0,-17.0


#### answering the question

In [33]:
# i'll use a different approach from the previous exercise
window = ['Season']
window = Window.partitionBy('Season').orderBy(col('TotalPoints').desc(), col('GoalDifferentials'))

table = home.join(away, ['Season', 'Team'], 'inner') \
            .withColumn('GoalsScored', col('HomeScoredGoals') + col('AwayScoredGoals')) \
            .withColumn('GoalsAgainst', col('HomeAgainstGoals') + col('AwayAgainstGoals')) \
            .withColumn('GoalDifferentials', col('GoalsScored') + col('GoalsAgainst')) \
            .withColumn('Wins', col('TotalHomeWin') + col('TotalAwayWin')) \
            .withColumn('Losses', col('TotalHomeLoss') + col('TotalAwayLoss')) \
            .withColumn('Tie', col('TotalHomeTie') + col('TotalAwayTie')) \
            .withColumn('TotalPoints', col('Wins') * 3 + col('Tie')) \
            .drop('TotalHomeWin', 'TotalHomeLoss', 'TotalHomeTie', 'TotalAwayWin', 'TotalAwayLoss', 'TotalAwayTie') \
            .drop('HomeScoredGoals', 'HomeAgainstGoals', 'AwayScoredGoals', 'AwayAgainstGoals') \
            .withColumn('TeamPosition', rank().over(window))

table_df = table.filter((col('TeamPosition') == 17) | (col('TeamPosition') == 18)).orderBy(desc('Season')).toPandas()
table_df

,Season,Team,GoalsScored,GoalsAgainst,GoalDifferentials,Wins,Losses,Tie,TotalPoints,TeamPosition
0,2016,Ingolstadt,36.0,57.0,93.0,8,18,8,32,17
1,2016,Darmstadt,28.0,63.0,91.0,7,23,4,25,18
2,2015,Stuttgart,50.0,75.0,125.0,9,19,6,33,17
3,2015,Hannover,31.0,62.0,93.0,7,23,4,25,18
4,2014,Freiburg,36.0,47.0,83.0,7,14,13,34,17
5,2014,Paderborn,31.0,65.0,96.0,7,17,10,31,18
6,2013,Nurnberg,37.0,70.0,107.0,5,18,11,26,17
7,2013,Braunschweig,29.0,60.0,89.0,6,21,7,25,18
8,2012,Fortuna Dusseldorf,39.0,57.0,96.0,7,18,9,30,17
9,2012,Greuther Furth,26.0,60.0,86.0,4,21,9,21,18
